# Model testing for ViT

This notebook will be structered in two sections

**section 1:** the plaintext model testing first the tensorboard will be checked then the accuracy report

**section 2:** the encrypted images first the data will be tested on the validation set to check the accuracy of the model then the test set will be used for checking the similarity metricies which are accuracy precision recall false positive rate (FPR)  

## Section 1 Plaintext model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%load_ext tensorboard


In [ ]:
from sklearn.metrics import accuracy_score
try:
    import timm

except ModuleNotFoundError:
    !pip install timm
    import timm

In [ ]:
!mkdir data
!unzip /content/drive/MyDrive/Bustati/potato.zip -d data

### 1.1 ViT Plaintext

In [ ]:
%tensorboard --logdir drive/MyDrive/Bustati/Bustati/runs

In [ ]:
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Create an instance of the model
model = timm.create_model("vit_base_patch16_224", num_classes=3)

# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/best_model.pth")

# Extract relevant information
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights



In [ ]:
import os
data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg)

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_preds = model(X).argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)




In [ ]:
if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_hat))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.title("ViT Plaintext ")
plt.savefig("ViT Plaintext conf-mat.svg",format="svg")
plt.show()

**The results are satisfying**

### 1.2 VGG-16

In [ ]:
%tensorboard --logdir drive/MyDrive/Bustati/Bustati/vgg_plaintext

In [ ]:
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Create an instance of the model
model = timm.create_model("vgg16", num_classes=3)

# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/vgg16_plaintext.pth")

# Extract relevant information
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights

import os
data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg)

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_preds = model(X).argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)

if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()


from sklearn.metrics import classification_report
print(classification_report(y_true,y_hat))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.title("VGG-16 Plaintext ")
plt.savefig("VGG-16 Plaintext conf-mat.svg",format="svg")
plt.show()

## Section 2 encrypted models

### 2.1 ViT model

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Bustati/Bustati/encrypted_run

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import torch
from torchvision import transforms

class ImageEncryption:
    """
    Class for image encryption using a combination of intra-block texture encryption
    and color encryption.

    Args:
        m (int): The number of subblocks in each dimension.

    Attributes:
        m (int): The number of subblocks in each dimension.

    Methods:
        image_encryption(image_path):
            Encrypts an image using intra-block texture encryption and color encryption.

        intrablock_texture_encryption(subblock):
            Encrypts a subblock using intra-block texture encryption.

        color_encryption(subblock, i):
            Encrypts a subblock using color encryption.
    """
    def __init__(self, m):
        """
        Initializes the ImageEncryption object.

        Args:
            m (int): The number of subblocks in each dimension.
        """
        self.m = m

    def image_encryption(self, image_path):
        """
        Encrypts an image using intra-block texture encryption and color encryption.

        Args:
            image_path (str): The path to the input image file.

        Returns:
            tuple: A tuple containing the encrypted image, image subkeys, and block keys.
        """
        # Load image using cv2.imread
        open_cv_image = np.array(image_path)
        # Convert RGB to BGR
        open_cv_image = open_cv_image[:, :, ::-1].copy()
        img = cv2.resize(open_cv_image,(224,224))
        height, width = img.shape[:2]  # Get height and width using img.shape
        w_subblock = width // self.m
        h_subblock = height // self.m

        image_subblocks = []
        for i in range(self.m):
            for j in range(self.m):
                x1 = i * w_subblock
                y1 = j * h_subblock
                x2 = x1 + w_subblock
                y2 = y1 + h_subblock
                subblock = img[y1:y2, x1:x2]  # Crop using array slicing
                image_subblocks.append(subblock)

        encrypted_image_subblocks = []
        key_image_subblocks = []
        for subblock in image_subblocks:
            encrypted_subblock, subK = self.intrablock_texture_encryption(np.array(subblock))
            encrypted_image_subblocks.append(encrypted_subblock)
            key_image_subblocks.append(subK)

        img_sub_key = np.empty(len(encrypted_image_subblocks))
        for i in range(len(encrypted_image_subblocks)-1, -1, -1):
            n = random.randint(0, i)
            encrypted_image_subblocks[i], encrypted_image_subblocks[n] = encrypted_image_subblocks[n], encrypted_image_subblocks[i]
            key_image_subblocks[i], key_image_subblocks[n] = key_image_subblocks[n], key_image_subblocks[i]
            img_sub_key[i] = n

        for i in range(len(encrypted_image_subblocks)):
            encrypted_image_subblocks[i] = self.color_encryption(encrypted_image_subblocks[i], i)

        original_image = np.zeros((height, width, 3))
        for i in range(self.m):
            for j in range(self.m):
                x1 = i * w_subblock
                y1 = j * h_subblock
                x2 = x1 + w_subblock
                y2 = y1 + h_subblock
                original_image[y1:y2, x1:x2] = encrypted_image_subblocks[i * self.m + j]
        encrypted_image_subblocks = original_image

        return encrypted_image_subblocks.astype(np.uint8), img_sub_key, key_image_subblocks

    def intrablock_texture_encryption(self, subblock):
        """
        Encrypts a subblock using intra-block texture encryption.

        Args:
            subblock (numpy.ndarray): The input subblock to be encrypted.

        Returns:
            tuple: A tuple containing the encrypted subblock and a dictionary of subkeys.
        """
        lr, lg, lb = cv2.split(subblock)

        # Flatten the color channels
        lr_flat = lr.flatten()
        lg_flat = lg.flatten()
        lb_flat = lb.flatten()

        # Initialize empty key lists for each channel
        sub_img_r_key = np.empty(lr_flat.shape)
        sub_img_g_key = np.empty(lr_flat.shape)
        sub_img_b_key = np.empty(lr_flat.shape)

        # Random shuffling with key generation
        for i in range(len(lb_flat)-1, -1, -1):
            r = random.randint(0, i)
            sub_img_r_key[i] = r
            g = random.randint(0, i)
            sub_img_g_key[i] = g
            b = random.randint(0, i)
            sub_img_b_key[i] = b

            lr_flat[i], lr_flat[r] = lr_flat[r], lr_flat[i]
            lg_flat[i], lg_flat[g] = lg_flat[g], lg_flat[i]
            lb_flat[i], lb_flat[b] = lb_flat[b], lb_flat[i]

        # Reshape flattened arrays to original shape
        lr_flat = lr_flat.reshape(lr.shape)
        lg_flat = lg_flat.reshape(lg.shape)
        lb_flat = lb_flat.reshape(lb.shape)

        # Merge that to become an RGB image
        encrypted_subblock = cv2.merge((lr_flat, lg_flat, lb_flat))

        # Create subK dictionary to store keys
        subK = {
            "sub_img_r_key": sub_img_r_key,
            "sub_img_g_key": sub_img_g_key,
            "sub_img_b_key": sub_img_b_key
        }
        return encrypted_subblock, subK

    def color_encryption(self, subblock, i):
        """
        Encrypts a subblock using color encryption.

        Args:
            subblock (numpy.ndarray): The input subblock to be encrypted.
            i (int): An index used in the encryption process.

        Returns:
            numpy.ndarray: The encrypted subblock.
        """
        er, eg, eb = cv2.split(subblock)
        er_flat = er.flatten()
        eg_flat = eg.flatten()
        eb_flat = eb.flatten()

        eb_flat_ = np.asarray([(e + i / self.m * 256 / self.m) % 256 for e in er_flat]).astype(int)
        er_flat_ = np.asarray([(e + (i % self.m) * 256 / self.m) % 256 for e in eg_flat]).astype(int)
        eg_flat_ = np.asarray([(e + (i / self.m + i % self.m) * 128 / self.m) % 256 for e in eb_flat]).astype(int)

        er_flat_ = er_flat_.reshape(er.shape)
        eg_flat_ = eg_flat_.reshape(eg.shape)
        eb_flat_ = eb_flat_.reshape(eb.shape)

        encrypted_subblock = cv2.merge((er_flat_, eg_flat_, eb_flat_))

        return encrypted_subblock.astype(np.uint8)


class ImageEncryptionTransform(object):
    def __init__(self, m):
        super().__init__()
        self.m = m
        self.image_encryption = ImageEncryption(m)  # Create an instance of your class

    def __call__(self, img):
        # Process the image using the image_encryption object
        encrypted_img, _, _ = self.image_encryption.image_encryption(img)
        encrypted_img = encrypted_img.astype(np.float32)
        encrypted_img/=255.0
        encrypted_img = torch.from_numpy(encrypted_img)
        return encrypted_img.permute(2,0,1)


In [ ]:
import os
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report

def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

# Create an instance of the model
model = timm.create_model("vit_base_patch16_224", num_classes=3)

# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/best_model_encrypted.pth")

# Extract relevant information
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights
transform = transforms.Compose([
    ImageEncryptionTransform(m=8),  # Apply encryption with desired number of subblocks
])

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_logits = model(X).softmax(axis=1)
        y_preds = y_logits.argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)

if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()


print(classification_report(y_true,y_hat))

In [ ]:
print(classification_report(y_true,y_hat))


In [ ]:
accuracy_score(y_true,y_hat)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.title("ViT 8 Blocks ")
plt.savefig("ViT 8 Blocks conf-mat.svg",format="svg")

plt.show()

### 2.2 VGG-16

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Bustati/Bustati/encrypted_vgg

In [ ]:
import os
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report

def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

# Create an instance of the model
model = timm.create_model("vgg16", num_classes=3)

# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/best_vgg.pth")

# Extract relevant information
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights
transform = transforms.Compose([
    ImageEncryptionTransform(m=8),  # Apply encryption with desired number of subblocks
])

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_logits = model(X).softmax(axis=1)
        y_preds = y_logits.argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)

if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()


print(classification_report(y_true,y_hat))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.show()

## Section 3 Encrypted domain with one block

### 3.1 ViT model

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Bustati/Bustati/one_block_vit

In [ ]:
import os
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report

def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

# Create an instance of the model
model = timm.create_model("vit_base_patch16_224", num_classes=3)

# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/one_block_vit.pth")

# Extract relevant information
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights
transform = transforms.Compose([
    ImageEncryptionTransform(m=1),  # Apply encryption with desired number of subblocks
])

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_logits = model(X).softmax(axis=1)
        y_preds = y_logits.argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)

if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()


print(classification_report(y_true,y_hat))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.title("ViT 1 Block ")
plt.savefig("ViT 1 Block conf-mat.svg",format="svg")

plt.show()

### 3.2 VGG-16

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Bustati/Bustati/one_block_vgg

In [ ]:
import os
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report

def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

# Create an instance of the model
model = timm.create_model("vgg16", num_classes=3)

# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/one_block_vgg.pth")

# Extract relevant information
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights
transform = transforms.Compose([
    ImageEncryptionTransform(m=1),  # Apply encryption with desired number of subblocks
])

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_logits = model(X).softmax(axis=1)
        y_preds = y_logits.argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)

if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()


print(classification_report(y_true,y_hat))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.title("VGG-16 1 Block ")
plt.savefig("VGG-16 1 Block conf-mat.svg",format="svg")

plt.show()

## Section 4 ONN

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Bustati/Bustati/onn_vgg

In [ ]:
 !pip install git+https://github.com/junaidmalik09/fastonn.git
 !git clone https://github.com/junaidmalik09/fastonn.git

In [ ]:
# get the files of VLPSO
import sys
from fastonn.osl import *
from fastonn.utils import *
from fastonn import OpNetwork,utils,OpTier,OpBlock,Trainer
from fastonn.utils import get_dataset_with_folds,ONNDataset
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn.functional as F
sys.path.insert(0,'/content/fastonn')
nodal = [mul,cubic,sine,expp,sinh,sinc2.apply,chirp]
act = [tanh,lincut]
pool = [summ]
OPLIB = getOPLIB(nodal,pool,act)

In [ ]:
import os
import timm
import torch
from pathlib import Path
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report

def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

# Create an instance of the model
onnmodel = timm.create_model('vgg16',pretrained=True,num_classes=3)

in_channels=3
tier_sizes=[3,16,3]
kernel_sizes=[21,3,16]
operators=[[1],[3],[6]]
sampling_factors=[2,-2,1]
OPLIB=OPLIB
pad=-1

for i in range(len(tier_sizes)):
  if i==0: onnmodel.add_module(str(i),OpTier(in_channels,tier_sizes[i],kernel_sizes[i],operators[i],OPLIB))
  else: onnmodel.add_module(str(i),OpTier(tier_sizes[i-1],tier_sizes[i],kernel_sizes[i],operators[i],OPLIB))

onnmodel.add_module('flatten final', nn.Flatten())
onnmodel.add_module('classifier final', nn.Linear(tier_sizes[-1] * 224*224, 3))




# Load the state_dict
checkpoint = torch.load("/content/drive/MyDrive/Bustati/Bustati/onn_vgg.pth")

# Extract relevant information
onnmodel.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
optimizer_state_dict = checkpoint['optimizer_state_dict']

# The model is now loaded with the trained weights
transform = transforms.Compose([
    ImageEncryptionTransform(m=8),  # Apply encryption with desired number of subblocks
])

data_path = Path("/content/data")
test_path = data_path/"test"

test_data = ImageFolder(
    test_path,
    transform=transform,
)
data_loader = DataLoader(
    test_data,
    shuffle=False,
    num_workers=int(os.cpu_count()),
    batch_size=32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
onnmodel.apply(reset_function_generic)
onnmodel = onnmodel.to(device)
onnmodel.eval()
y_true = []
y_hat = []
with torch.inference_mode():
    for X,y in data_loader:
        X,y=X.to(device),y.to(device)
        y_logits = onnmodel(X).softmax(axis=1)
        y_preds = y_logits.argmax(axis=1)
        y_true.append(y)
        y_hat.append(y_preds)
    y_true=torch.cat(y_true,0)
    y_hat=torch.cat(y_hat,0)

if y_true.device != "cpu":
    y_true = y_true.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
else:
    y_true = y_true.numpy()
    y_hat = y_hat.numpy()


print(classification_report(y_true,y_hat))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = confusion_matrix(y_true,y_hat)
disp = ConfusionMatrixDisplay(cf,display_labels=np.array(test_data.classes))
disp.plot()
plt.title("VGG-16 with ONN ")
plt.savefig("VGG-16 with ONN conf-mat.svg",format="svg")

plt.show()